In [2]:
# !pip install xgboost

  Using cached xgboost-2.1.0-py3-none-manylinux_2_28_x86_64.whl.metadata (2.1 kB)
  Using cached nvidia_nccl_cu12-2.22.3-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
Using cached xgboost-2.1.0-py3-none-manylinux_2_28_x86_64.whl (153.9 MB)
Using cached nvidia_nccl_cu12-2.22.3-py3-none-manylinux2014_x86_64.whl (190.9 MB)


In [4]:
import os
import re
import sys
import math
import json
import time
import joblib
import pickle
import warnings
import boto3
import botocore
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from IPython.display import Image
from IPython.display import display
from IPython.display import FileLink, FileLinks
from platformdirs import site_config_dir, user_config_dir
from time import gmtime, strftime

from sagemaker import Session
from sagemaker import get_execution_role
from sagemaker.experiments.run import Run, load_run
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.spark.processing import PySparkProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.remote_function import remote, RemoteExecutor
from sagemaker.tuner import (
    CategoricalParameter, ContinuousParameter,
    HyperparameterTuner, IntegerParameter,
)

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [5]:
print(pd.__version__)
print(xgb.__version__)
print(sagemaker.__version__)

2.2.2
2.1.0
2.215.0


In [6]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 30)

# warnings.filterwarnings("ignore")

In [7]:
boto_session = boto3.Session()
client_sagemaker = boto_session.client("sagemaker")
client_s3 = boto_session.client("s3")
sagemaker_session = sagemaker.Session()

In [8]:
%store -r 

%store

Stored variables and their in-db values:
baseline_s3_url                         -> 's3://sagemaker-us-east-1-891377318910/workshop/da
bucket_name                             -> 'sagemaker-us-east-1-891377318910'
bucket_prefix                           -> 'from-idea-to-production/xgboost'
dataset_feature_group_name              -> 'from-idea-to-prod-06-16-17-29'
dataset_file_local_path                 -> 'data/bank-additional/bank-additional-full.csv'
dataset_raw                             -> 'bank-additional-full.csv'
domain_id                               -> 'd-ehxji4qaadry'
evaluation_s3_url                       -> 's3://sagemaker-us-east-1-891377318910/from-idea-t
experiment_name                         -> 'itau-experiment-16-22-20-13'
feature_store_bucket_prefix             -> 'from-idea-to-prod/feature-store'
initialized                             -> True
input_s3_url                            -> 's3://sagemaker-us-east-1-891377318910/workshop/da
loca_transformed_path       

In [9]:
local_train_data = f"{loca_transformed_path}/train.csv"
local_validation_data = f"{loca_transformed_path}/validation.csv"
local_testx_data = f"{loca_transformed_path}/test_x.csv"
local_testy_data = f"{loca_transformed_path}/test_y.csv"

key_prefix = f"{s3_prefix}/data/transformed"

client_s3.download_file(bucket_name, f"{key_prefix}/train/train.csv", local_train_data)
client_s3.download_file(bucket_name, f"{key_prefix}/validation/validation.csv", local_validation_data)
client_s3.download_file(bucket_name, f"{key_prefix}/test/test_x.csv", local_testx_data)
client_s3.download_file(bucket_name, f"{key_prefix}/test/test_y.csv", local_testy_data)

In [10]:
# train_data = pd.read_csv(local_train_data, engine="pyarrow", dtype_backend="pyarrow", header=None)
# validation_data = pd.read_csv(local_validation_data, engine="pyarrow", dtype_backend="pyarrow", header=None)
# test_data = pd.read_csv(local_test_data, engine="pyarrow", dtype_backend="pyarrow", header=None)

# train_data = np.genfromtxt(local_train_data, delimiter=',', names=None)
# train_data = np.genfromtxt(local_train_data, delimiter=',', names=None)
# train_data = np.genfromtxt(local_train_data, delimiter=',', names=None)

# train_data = pd.read_csv(local_train_data, engine="pyarrow", dtype_backend="pyarrow")
# validation_data = pd.read_csv(local_validation_data, engine="pyarrow", dtype_backend="pyarrow")
# test_data = pd.read_csv(local_test_data, engine="pyarrow", dtype_backend="pyarrow")

train_data = pd.read_csv(local_train_data)
validation_data = pd.read_csv(local_validation_data)
testx_data = pd.read_csv(local_testx_data)
testy_data = pd.read_csv(local_testy_data)

In [11]:
test_data = pd.concat([testy_data, testx_data], axis=1)

In [12]:
test_data

,y,campaign,pdays,previous,no_previous_contact,not_working,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,0.018182,1.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
1,0,0.072727,1.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
2,0,0.127273,1.0,0.0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0
3,1,0.000000,1.0,0.0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
4,0,0.000000,1.0,0.0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6174,0,0.000000,1.0,0.0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
6175,0,0.000000,1.0,0.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
6176,1,0.000000,1.0,0.0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
6177,1,0.018182,1.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0


# Model training

## Option 1 - local:

In [13]:
train_features = train_data.drop(target_col, axis=1)
train_label = pd.DataFrame(train_data[target_col])

In [14]:
dtrain = xgb.DMatrix(train_features, label=train_label)

In [15]:
hyperparams = {
    "max_depth": 5,
    "eta": 0.5,
    "alpha": 2.5,
    "objective": "binary:logistic",
    "subsample" : 0.8,
    "colsample_bytree" : 0.8,
    "min_child_weight" : 3
}

num_boost_round = 150
nfold = 3
early_stopping_rounds = 10

In [16]:
# Cross-validate on training data
cv_results = xgb.cv(
    params=hyperparams,
    dtrain=dtrain,
    num_boost_round=num_boost_round,
    nfold=nfold,
    early_stopping_rounds=early_stopping_rounds,
    metrics=["auc"],
    seed=10,
)

In [17]:
# metrics_data = {
#     "binary_classification_metrics": {
#         "validation:auc": {
#             "value": cv_results.iloc[-1]["test-auc-mean"],
#             "standard_deviation": cv_results.iloc[-1]["test-auc-std"]
#         },
#         "train:auc": {
#             "value": cv_results.iloc[-1]["train-auc-mean"],
#             "standard_deviation": cv_results.iloc[-1]["train-auc-std"]
#         },
#     }
# }

In [19]:
print(f"Cross-validated train-auc:{cv_results.iloc[-1]['train-auc-mean']:.2f}")
print(f"Cross-validated validation-auc:{cv_results.iloc[-1]['test-auc-mean']:.2f}")

Cross-validated train-auc:0.81
Cross-validated validation-auc:0.77


In [20]:
cv_results

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.724214,0.000733,0.723305,0.001910
1,0.749119,0.006117,0.747220,0.006352
2,0.771417,0.002461,0.763940,0.003854
3,0.778388,0.004207,0.766431,0.002843
4,0.784141,0.002509,0.770384,0.000347
5,0.786868,0.002554,0.771433,0.001112
6,0.790005,0.001921,0.771800,0.000714
7,0.791822,0.001947,0.772963,0.001033
8,0.794117,0.002245,0.773545,0.001295
9,0.796234,0.001638,0.774439,0.002012


In [21]:
test_features = test_data.drop(target_col, axis=1)
test_label = pd.DataFrame(test_data[target_col])
dtest = xgb.DMatrix(test_features, label=test_label)

## Option 2 - local:

In [22]:
train_features = train_data.drop(target_col, axis=1)
train_label = pd.DataFrame(train_data[target_col])

In [23]:
dtrain = xgb.DMatrix(train_features, label=train_label)

In [24]:
run_suffix = strftime('%Y-%m-%M-%S', gmtime())
run_name = f"training-{run_suffix}"

with Run(experiment_name=experiment_name,
         run_name=run_name,
         run_display_name="training",
         sagemaker_session=sagemaker_session) as run:
    run.log_parameters(
        {
            "train": 0.7,
            "validate": 0.2,
            "test": 0.1
        }
    )
    # Log input dataset metadata and output
    run.log_artifact(name="dataset", value="./data/bank-additional/bank-additional-full.csv", media_type="text/csv", is_output=False)
    run.log_artifact(name="train-csv", value="./data/train.csv", media_type="text/csv")
    run.log_artifact(name="validation-csv", value="./data/validation.csv", media_type="text/csv")
    run.log_artifact(name="test-csv", value="./data/test.csv", media_type="text/csv")

In [40]:
# in the production code you need to use the unique ids
run_suffix = strftime('%Y-%m-%M-%S', gmtime())

# Train the model for different max_depth values
for i, d in enumerate([2, 5, 10, 15, 20]):
    hyperparams["max_depth"] = d

    print(f"Fit estimator with max_depth={d}")
    run_name = f"training-{i}-{run_suffix}"

    with Run(experiment_name=experiment_name,
             run_name=run_name,
             run_display_name=f"max-depth-{d}",
             sagemaker_session=sagemaker_session) as run:

        # Train the model
        model = (
            xgb.train(
                params=hyperparams,
                dtrain=dtrain,
                evals = [(dtrain,'train'), (dtest,'eval')],
                num_boost_round=num_boost_round,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval = 0
            )
        )

        # Calculate metrics
        test_auc = roc_auc_score(test_label, model.predict(dtest))
        train_auc = roc_auc_score(train_label, model.predict(dtrain))

        # Log metrics to the run
        run.log_parameters(hyperparams)
        run.log_metric(name="test_auc", value=test_auc, step=d)
        run.log_metric(name="train_auc", value=train_auc, step=d)

        # time.sleep(8) # wait until resource tags are propagated to the run

        print(f"Test AUC: {test_auc:.4f} | Train AUC: {train_auc:.4f}")

Fit estimator with max_depth=2
Test AUC: 0.7539 | Train AUC: 0.7805
Fit estimator with max_depth=5
Test AUC: 0.7484 | Train AUC: 0.8095
Fit estimator with max_depth=10
Test AUC: 0.7486 | Train AUC: 0.8524
Fit estimator with max_depth=15
Test AUC: 0.7466 | Train AUC: 0.8733
Fit estimator with max_depth=20
Test AUC: 0.7401 | Train AUC: 0.8809


## Option 3 - local:

In [25]:
# get training container uri
training_image = sagemaker.image_uris.retrieve(
    framework="xgboost", 
    region=region,
    version="1.5-1"
)

print(training_image)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1


In [43]:
s3_input_train = sagemaker.inputs.TrainingInput(train_s3_url, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(validation_s3_url, content_type='csv')

In [17]:
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

# Define where the training job stores the model artifact
output_s3_url = f"s3://{bucket_name}/{s3_prefix}/output"
print(output_s3_url)

%store output_s3_url

s3://sagemaker-us-east-1-891377318910/workshop/output
Stored 'output_s3_url' (str)


In [45]:
# Instantiate an XGBoost estimator object
estimator = sagemaker.estimator.Estimator(
    image_uri=training_image,  # XGBoost algorithm container
    instance_type=train_instance_type,  # type of training instance
    instance_count=train_instance_count,  # number of instances to be used
    role=sm_role,  # IAM execution role to be used
    max_run=20 * 60,  # Maximum allowed active runtime
    # use_spot_instances=True,  # Use spot instances to reduce cost
    # max_wait=30 * 60,  # Maximum clock time (including spot delays)
    output_path=output_s3_url, # S3 location for saving the training result
    sagemaker_session=sagemaker_session, # Session object which manages interactions with SageMaker API and AWS services
    base_job_name="from-idea-to-prod-training", # Prefix for training job name
    keep_alive_period_in_seconds=60,            # use the warm pool feature
)

In [46]:
# define its hyperparameters
estimator.set_hyperparameters(
    num_round=150, # the number of rounds to run the training
    max_depth=3, # maximum depth of a tree
    eta=0.5, # step size shrinkage used in updates to prevent overfitting
    alpha=2.5, # L1 regularization term on weights
    objective="binary:logistic",
    eval_metric="auc", # evaluation metrics for validation data
    subsample=0.8, # subsample ratio of the training instance
    colsample_bytree=0.8, # subsample ratio of columns when constructing each tree
    min_child_weight=3, # minimum sum of instance weight (hessian) needed in a child
    early_stopping_rounds=10, # the model trains until the validation score stops improving
    verbosity=1, # verbosity of printing messages
)

In [47]:
training_inputs = {'train': s3_input_train, 'validation': s3_input_validation}

In [48]:
try:
    run_suffix = strftime('%Y-%m-%M-%S', gmtime())
    run_name = f"container-training-{run_suffix}"

    with Run(experiment_name=experiment_name,
             run_name=run_name,
             run_display_name="container-training",
             sagemaker_session=sagemaker_session
            ) as run:
        
        estimator.fit(
            training_inputs,
            wait=True,
            logs=False,
        )

except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'AccessDeniedException':
        print(f"Ignore AccessDeniedException: {e.response['Error']['Message']} because of the slow resource tag auto propagation")
    else:
        raise e

INFO:sagemaker:Creating training-job with name: from-idea-to-prod-training-2024-06-18-01-21-02-771



2024-06-18 01:21:06 Starting - Starting the training job..
2024-06-18 01:21:20 Starting - Preparing the instances for training....
2024-06-18 01:21:47 Downloading - Downloading input data....
2024-06-18 01:22:12 Downloading - Downloading the training image......
2024-06-18 01:22:48 Training - Training image download completed. Training in progress...
2024-06-18 01:23:03 Uploading - Uploading generated training model..
2024-06-18 01:23:16 Completed - Instances not retained as a result of warmpool resource limits being exceeded


## Option 4:

In [18]:
# get training container uri
training_image = sagemaker.image_uris.retrieve(
    framework="xgboost", 
    region=region,
    version="1.5-1"
)

print(training_image)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1


In [19]:
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

# Define where the training job stores the model artifact
output_s3_url = f"s3://{bucket_name}/{s3_prefix}"
print(output_s3_url)

%store output_s3_url

s3://sagemaker-us-east-1-891377318910/workshop/output
Stored 'output_s3_url' (str)


In [20]:
# Instantiate an XGBoost estimator object
estimator = sagemaker.estimator.Estimator(
    image_uri=training_image,  # XGBoost algorithm container
    instance_type=train_instance_type,  # type of training instance
    instance_count=train_instance_count,  # number of instances to be used
    role=sm_role,  # IAM execution role to be used
    max_run=20 * 60,  # Maximum allowed active runtime
    # use_spot_instances=True,  # Use spot instances to reduce cost
    # max_wait=30 * 60,  # Maximum clock time (including spot delays)
    output_path=output_s3_url, # S3 location for saving the training result
    sagemaker_session=sagemaker_session, # Session object which manages interactions with SageMaker API and AWS services
    base_job_name="itau-training", # Prefix for training job name
    keep_alive_period_in_seconds=60,            # use the warm pool feature
    deploy_mode=False
)

In [21]:
grid_hiperparametros = {
    'max_depth': IntegerParameter(3, 9),
    'eta': ContinuousParameter(0.1, 0.8),
    'gamma': ContinuousParameter(0.0, 0.8),
    'min_child_weight': IntegerParameter(1, 6),
    'subsample': ContinuousParameter(0.5, 1),
    'alpha': ContinuousParameter(0, 1.0),
    'lambda': ContinuousParameter(0, 0.1),
    'num_round': IntegerParameter(100, 101),
    'colsample_bytree': ContinuousParameter(0, 1),
}

In [28]:
# Cria o otimizador
estimator_tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name="validation:rmse",
    objective_type="Minimize",
    hyperparameter_ranges=grid_hiperparametros,
    max_jobs=28,
    max_parallel_jobs=8,
    # strategy="Bayesian",
    # early_stopping_type="Auto",
    # base_tuning_job_name="from-idea-to-prod-hpo",
)

In [29]:
# Cria o input de treino
s3_input_train = sagemaker.inputs.TrainingInput(train_s3_url, content_type='text/csv')

# Cria o input de validação
s3_input_validation = sagemaker.inputs.TrainingInput(validation_s3_url, content_type='text/csv')

In [30]:
%%time
# max_parallel_jobs=8
estimator_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation}, wait=True, logs=False)

INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-240619-0114


...............................................................!
CPU times: user 329 ms, sys: 22.7 ms, total: 352 ms
Wall time: 5min 27s


In [31]:
estimator_tuner.stop_tuning_job()

INFO:sagemaker:Stopping tuning job: sagemaker-xgboost-240619-0114
INFO:sagemaker:Tuning job: sagemaker-xgboost-240619-0114 is already stopped or not running.


In [32]:
best_training_job_name = estimator_tuner.best_training_job()
best_training_job_name

'sagemaker-xgboost-240619-0114-024-490af007'

In [33]:
# Descrevendo o melhor job de treinamento para obter os hiperparâmetros
best_training_job_info = sagemaker_session.describe_training_job(job_name=best_training_job_name)

In [ ]:
# best_training_job_info

In [34]:
best_hyperparameters = best_training_job_info['HyperParameters']

In [35]:
print("Melhores Hiperparâmetros:")
print(best_hyperparameters)

Melhores Hiperparâmetros:
{'_tuning_objective_metric': 'validation:rmse', 'alpha': '0.0024068039604222104', 'eta': '0.10396137822547491', 'gamma': '0.15168194547649314', 'lambda': '0.022201711333640198', 'max_depth': '4', 'min_child_weight': '4', 'num_round': '101', 'subsample': '0.570630576173532'}


In [37]:
# Set dos melhores hiperparâmetros
estimator.set_hyperparameters(**best_hyperparameters)

In [39]:
# Treinando a versão final do modelo com os melhores hiperparâmetros
job_name = "itau-best-xgboost-1"
estimator.fit({'train': s3_input_train, 'validation': s3_input_validation}, job_name=job_name)

INFO:sagemaker:Creating training-job with name: itau-best-xgboost-1


2024-06-19 01:21:57 Starting - Starting the training job...
2024-06-19 01:22:14 Starting - Preparing the instances for training...
2024-06-19 01:22:40 Downloading - Downloading input data...
2024-06-19 01:23:06 Downloading - Downloading the training image...
2024-06-19 01:23:41 Training - Training image download completed. Training in progress../miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2024-06-19 01:23:53.083 ip-10-2-201-181.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-19 01:23:53.105 ip-10-2-201-181.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-06-19:01:23:53:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-19:01:23:53:INFO] Failed to parse hyperparameter _tuning_objective_metri

# Evaluate the Model

In [ ]:
if estimator._current_job_name:
    training_job_name = estimator._current_job_name
    print("Job name:", training_job_name)

In [ ]:
metrics = None

while not metrics:
    metrics = sagemaker_client.describe_training_job(TrainingJobName=training_job_name).get("FinalMetricDataList")
    if not metrics:
        print(f"Training job {training_job_name} hasn't finished yet!")
        time.sleep(10)
    
train_auc = float([m['Value'] for m in metrics if m['MetricName'] == 'train:auc'][0])
validate_auc = float([m['Value'] for m in metrics if m['MetricName'] == 'validation:auc'][0])

print(f"Train-auc:{train_auc:.2f}, Validate-auc:{validate_auc:.2f}")

In [ ]:
# %store training_job_name

In [ ]:
estimator.model_data

In [47]:
client_s3.download_file(bucket_name, f'{s3_prefix}/output/itau-best-xgboost-1/output/model.tar.gz', 'model.tar.gz')
#"s3://sagemaker-us-east-1-891377318910/workshop/output/itau-best-xgboost-1/output/model.tar.gz"

In [48]:
!tar -zxvf model.tar.gz

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
xgboost-model


In [52]:
# Carrega o modelo
# with open('xgboost-model', 'rb') as file:
#     xgboost_model = pickle.load(file)

model = xgb.Booster()
model.load_model('xgboost-model')

In [65]:
# Extrai os atrobutos e labels
X_test_xgb = test_data.drop(columns=['y'])
y_test_xgb = test_data['y']

In [66]:
# Converte o dataframe para objeto DMatrix
dtest_xgb = xgb.DMatrix(X_test_xgb)

In [67]:
# Faz as previsões com o modelo XGBoost
predictions_xgboost = model.predict(dtest_xgb)
predictions_xgboost_rounded = np.round(predictions_xgboost)

In [68]:
# Calcula as métricas de avaliação do XGBoost com dados de teste
xgboost_metrics = {
    'MSE (XGBoost)': mean_squared_error(y_test_xgb, predictions_xgboost_rounded),
    'MAE (XGBoost)': mean_absolute_error(y_test_xgb, predictions_xgboost_rounded),
    'RMSE (XGBoost)': np.sqrt(mean_squared_error(y_test_xgb, predictions_xgboost_rounded))
}

In [69]:
print(xgboost_metrics)

{'MSE (XGBoost)': 0.10276743809677942, 'MAE (XGBoost)': 0.10276743809677942, 'RMSE (XGBoost)': 0.32057360792301576}


In [75]:
# Cria o dataframe com dados reais e previstos
xgboost_results_df = pd.DataFrame({'Real': y_test_xgb, 'Previsto': predictions_xgboost_rounded})
xgboost_results_df.head(10)

,Real,Previsto
0,0,0.0
1,0,0.0
2,0,0.0
3,1,0.0
4,0,0.0
5,0,0.0
6,0,0.0
7,0,0.0
8,0,0.0
9,0,0.0


## Validate model

In [ ]:
train_instance_type = "ml.m5.xlarge"
job_name = "itau-best-xgboost-1"

transform_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_prefix}/transformed"
# model_name = f"from-idea-to-prod-transform-{strftime('%d-%H-%M-%S', gmtime())}"
model_name = "from-idea-to-prod-transform-23-01-19-21"
base_transform_job_name = "itau-demo-trasform"

### Option 1: create a batch transformer from the trained estimator

In [ ]:
transformer = estimator.transformer(
    instance_count=1,
    instance_type=train_instance_type,
    accept="text/csv",
    role=sm_role,
    output_path=transform_s3_url,
    model_name=model_name,
)

### Option 2: load a model from a training job

In [ ]:
print(training_job_name)
print(model_name)

In [ ]:
model = sagemaker_session.create_model_from_job(
    training_job_name=training_job_name, 
    name=model_name,
)

In [ ]:
transformer = sagemaker.transformer.Transformer(
    model_name=model,
    instance_count=1,
    instance_type=train_instance_type,
    accept="text/csv",
    assemble_with="Line",
    output_path=transform_s3_url,
    base_transform_job_name=base_transform_job_name,
    sagemaker_session=sagemaker_session,
)

In [ ]:
job_name = "itau-best-xgboost-1"
model_name = f"{job_name}-model",

In [ ]:
training_image = sagemaker.image_uris.retrieve(
    framework="xgboost", 
    region=region,
    version="1.5-1"
)

In [ ]:
model = Model(
    image_uri=training_image,        
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    name=model_name,
    sagemaker_session=sagemaker_session,
    role=sm_role,
)

### Run transform job

In [ ]:
print(test_s3_url)

In [ ]:
transform_job_name = f"{base_transform_job_name}-{strftime('%d-%H-%M-%S', gmtime())}"

try:
    run_suffix = strftime('%Y-%m-%M-%S', gmtime())
    run_name = f"batch-transform-{run_suffix}"

    with Run(experiment_name=experiment_name,
             run_name=run_name,
             run_display_name="batch-transform",
             sagemaker_session=sagemaker_session
            ) as run:
        transformer.transform(    
            data=f"{test_s3_url}/test_x.csv",
            content_type="text/csv",
            split_type="Line", 
            job_name=transform_job_name,
            wait=True,
            # experiment_config=experiment_config,
        )

except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'AccessDeniedException':
        print(f"Ignore AccessDeniedException: {e.response['Error']['Message']} because of the slow resource tag auto propagation")
    else:
        raise e